In [43]:
import logging
import os

In [8]:
from first_order import *
from typing_extensions import TypeVar
from util import custom_test

In [2]:
f, c, x = FunctionSymbol("F", 1), FunctionSymbol("C", 0), Variable("x")
some_interpretation = Interpretation(
    Structure(range(10), {f: lambda n: n + 1, c: 5}),
    {x: 3},
)

In [3]:
some_interpretation

Interpretation(structure=Structure(universe=range(0, 10), relations=mappingproxy({}), functions=mappingproxy({FunctionSymbol(name='F', arity=1): <function <lambda> at 0x7fff781939c0>, FunctionSymbol(name='C', arity=0): <function _constant_function.<locals>.<lambda> at 0x7fff78193a60>})), assignment={Variable(name='$x'): 3})

In [5]:
"""Week 5 assignment."""

from first_order import AtomicFormula, FunctionSymbol, Interpretation, Structure, Term, Variable
from typing_extensions import TypeVar
from util import custom_test

T = TypeVar("T")

f, c, x = FunctionSymbol("F", 1), FunctionSymbol("C", 0), Variable("x")
some_interpretation = Interpretation(
    Structure(range(10), {f: lambda n: n + 1, c: 5}),
    {x: 3},
)


def complicated_test(value: int) -> bool:
    return True


@custom_test((f(x), some_interpretation), 4)
@custom_test((f(x), some_interpretation), [3, 4, 5])
@custom_test((f(x), some_interpretation), complicated_test)
def evaluate_term(term: Term, interpretation: Interpretation[T]) -> T:
    """Evaluates a term to the element of the universe defined by it and the interpretation.

    Args:
        term: An arbitrary term using the same signature as the structure and variables from the assignment.
        interpretation: The corresponding interpretation.
    Returns:
        The value the term evaluates to.
    Example:
        Given `f(a, b)` and an interpretation where `f` is interpreted as addition and `a` and `b` as 1 and 2,
        this function should return `3`.
    """

    assert term.symbols <= interpretation.structure.symbols
    assert term.variables <= interpretation.assignment.keys()

    if isinstance(term, Variable):
        return interpretation.assignment[term]
    elif isinstance(term, FunctionTerm):
        return interpretation.structure.functions[term.function](
            *[evaluate_term(arg, interpretation) for arg in term.arguments]
        )
    else:
        raise TypeError


@custom_test(input=(f(x) == c, some_interpretation), output=False)
def evaluate_atom(atom: AtomicFormula, interpretation: Interpretation[Any]) -> bool:
    """Evaluates an atomic formula to its truth value.

    Args:
        atom: An atomic formula (Equality, RelationFormula, or TruthConstant).
        interpretation: An interpretation over the same signature as the atomic formula that also interprets all
            variables occurring in it.
    Returns:
        The truth value of the atomic formula under the given interpretation.
    Example:
        Given `a == b` and an interpretation where `a` and `b` are interpreted as 1 and 2,
        this function should return `False`.
    """

    assert atom.symbols <= interpretation.structure.symbols
    assert atom.free_variables <= interpretation.assignment.keys()

    if isinstance(atom, TruthConstant):
        return atom.value
    elif isinstance(atom, Equality):
        return evaluate_term(atom.first, interpretation) == evaluate_term(atom.second, interpretation)
    elif isinstance(atom, RelationFormula):
        return interpretation.structure.relations[atom.relation](
            *[evaluate_term(arg, interpretation) for arg in atom.arguments]
        )
    else:
        raise TypeError


NameError: name 'Any' is not defined

In [21]:
def evaluate_atom(atom, interpretation):
    """Evaluates an atomic formula to its truth value.

    Args:
        atom: An atomic formula (Equality, RelationFormula, or TruthConstant).
        interpretation: An interpretation over the same signature as the atomic formula that also interprets all
            variables occurring in it.
    Returns:
        The truth value of the atomic formula under the given interpretation.
    Example:
        Given `a == b` and an interpretation where `a` and `b` are interpreted as 1 and 2,
        this function should return `False`.
    """

    assert atom.symbols <= interpretation.structure.symbols
    assert atom.free_variables <= interpretation.assignment.keys()

    if isinstance(atom, TruthConstant):
        return atom.value
    elif isinstance(atom, Equality):
        return evaluate_term(atom.first, interpretation) == evaluate_term(atom.second, interpretation)
    elif isinstance(atom, RelationFormula):
        return interpretation.structure.relations[atom.relation](
            *[evaluate_term(arg, interpretation) for arg in atom.arguments]
        )
    else:
        raise TypeError


In [9]:
def evaluate_term(term, interpretation):
    """Evaluates a term to the element of the universe defined by it and the interpretation.

    Args:
        term: An arbitrary term using the same signature as the structure and variables from the assignment.
        interpretation: The corresponding interpretation.
    Returns:
        The value the term evaluates to.
    Example:
        Given `f(a, b)` and an interpretation where `f` is interpreted as addition and `a` and `b` as 1 and 2,
        this function should return `3`.
    """

    assert term.symbols <= interpretation.structure.symbols
    assert term.variables <= interpretation.assignment.keys()

    if isinstance(term, Variable):
        return interpretation.assignment[term]
    elif isinstance(term, FunctionTerm):
        return interpretation.structure.functions[term.function](
            *[evaluate_term(arg, interpretation) for arg in term.arguments]
        )
    else:
        raise TypeError


In [13]:
evaluate_term(f(x), some_interpretation)

4

In [14]:
t, v1, v2 = FunctionSymbol("Add", 2), Variable("v1"), Variable("v2")

In [37]:
test = Interpretation(
    Structure(range(10), {r: lambda n,m: n+1 == m, t: lambda a,b: a+b}),
    {v1: 2, v2: 3}
)

In [27]:
r = RelationSymbol("R", 2)

In [48]:
evaluate_atom(TruthConstant(1), test)

True

In [41]:
type(t(v1, v2))

first_order.FunctionTerm

In [32]:
evaluate_term(t(v1,v2), test)

6